In [ ]:
import warnings
warnings.filterwarnings('ignore', 'The iteration is not making good progress')
import math
from scipy.optimize import fsolve


def Sutton_method (SG, *data):
    
    Pres, Tres, Z_factor = data #automatic unpacking of arguments
    #-----------------------------------------------------------#
        
    y = (0.969469 + ((-1.349238) * ((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres)) + (1.443959 * pow (((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),2)) + ((-0.36860)*pow(((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),3))) + ((-0.107783) + ((-0.127013) * ((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres)) + (0.100828 * pow (((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),2)) + ((-0.012319)*pow(((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),3)))*((706 - 51.7 * SG + 11.1 * pow(SG,2))/Pres) + (0.0184810 + ((0.0523405) * ((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres)) + (-0.050688 * pow (((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),2)) + ((0.010870)*pow(((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),3)))*pow(((706 - 51.7 * SG + 11.1 * pow(SG,2))/Pres),2) + ((-0.000584) + ((-0.002146) * ((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres)) + (0.0020961 * pow (((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),2)) + ((-0.000459)*pow(((187 + 330.5 * SG + 71.5 * pow(SG,2))/Tres),3)))*pow(((706 - 51.7 * SG + 11.1 * pow(SG,2))/Pres),3) - Z_factor
        
    return y

def gas_gravity_calculator (Pres, Tres):
    
    #Pres, Tres should be inputted in psia and in Rannkine respectively 
    
    
    ##-----------------------------------------------------------------##
    A = [0.0001946, -0.027635, 0.136315, -0.238489, 0.105168, 3.44*10**8]
    SG2 = 10000000 ## Pre-assigning the variables not to confuse the code
    Sg_initial = 0.55   ## Pre-assigning the variables not to confuse the code
    MWair = 28.97      ## Molecular Weight of water in kg/kmole
    ##-----------------------------------------------------------------##
    
    while ((SG2 - Sg_initial) > 0.000000001):
        
        ## The input value SG is the specific gravity of gas Since methane is the main component of natural gas,
        ## its specific gravity was chosen as initial guess - by default which is 0.55
        print ("The value of guessed SG: {}".format (Sg_initial))
        
        ##---Reservoir Parameters-----------------##
        R = 8314 ## in J/ (kmole *K)
        Ro_gas_reservoir = 184 ##in kg/m3
        Tres_F = Tres - 460
        Presf = Pres/1000
        ##----------------------------------------##
        
        ##---Equations relating Z-factor to Gas Gravity -SG as the analytics of Standing and Katz---##
        f1 = Presf*(0.251*Sg_initial - 0.15)  - 0.202*Sg_initial + 1.106
        f2 = 1.4*math.exp(-0.0054*(Tres_F))
        f3 = A[0]*Presf**5 + A[1]*Presf**4 + A[2]*Presf**3 + A[3]*Presf**2 + A[4]*Presf
        f4 = (0.154 - 0.152*Sg_initial)*(Presf**(3.18*Sg_initial - 1.0)) * math.exp(-0.5*Presf) - 0.02
        f5 = 0.35 * ((0.6 - Sg_initial)*math.exp(-1.039*((Presf - 1.8)**2)))
        Z_factor = f1*((1/(1 + ((A[5]*Presf*10**(1.785*Sg_initial))/((Tres_F+460)**3.825)))) + f2*f3) + f4 + f5
        ##-----------------------------------------------------------------------------------------##
        
        
        T_res_Kelvin = (5/9)*(Tres - 491.67) + 273.15
        P_res_Pa = Pres/0.000145
        
        #Calculating the value of SG from equation of Standing (1981) and Tuned cooefficients
        #-----------------------------------------------------------#
        data = (Pres, Tres, Z_factor)
        SG = fsolve (Sutton_method, 0.8, args = data)
        #-----------------------------------------------------------#
        print ("Specific Gravity: {}" .format(SG))
        Sg_initial = Sg_initial + 0.001
        
        if SG - Sg_initial < 0.0001:
            break